# SOLID Principles

SOLID principles are a set of recommendations to design and write better
software systems. They were proposed by Robert C. Martin.

Their purposes are:
- Write more mantainable code, which it's easy to change.
- Write code which can be extended with new funcionalities easily.
- Write more readable code. 

SOLID is conformed by 5 principles:
- Single Responsibility Principle (SRP)
- Open Closed Principle (OCP)
- Liskov Substitution Principle (LSP)
- Interface Segregation Principle (ISP)
- Dependency Inversion Principle (DIP)

## 1. Single Responsibility Principle (SRP)

This principle stands that each class should have a unique responsibility.
Each class manages a specific part of the system and performs
very well on one concrete task.

As Robert said: "Gather together the things that change for the same reasons. Separate those things that change for different reasons."

### Example: User Registry in a database

- Without satisfying the SRP principle:

In [5]:
import bcrypt

# This class represents an user
class User:
    def __init__(self, email: str, password: str) -> None:
        self.email = email
        self.password = password
        # You can add here more info about the user...
        
    def __str__(self) -> str:
        return f"Email: {self.email} - Password: {self.password}"
        
# This class manages database operations
class DatabaseManager:
    
    # Here you will have tons of methods to manage database operations...
    
    @staticmethod
    def save_to_database(user: User) -> None:
        print(f"User {str(user)} has been saved on the database.")

# And finally here is a class that registers a user in the database
class UserRegistry:
    
    @staticmethod
    def createUser(email: str, password: str) -> None:
        """Method for creating a new user."""
        
        # First, we have to encrypt the user's password
        encrypted_password = bcrypt.hashpw(password.encode(), bcrypt.gensalt())
        
        # Then, we create the User object
        user = User(email, encrypted_password)
        
        # And finally we send it to the database
        DatabaseManager.save_to_database(user)
        
if  __name__ == "__main__":
    
    # Let's create and register a new user
    UserRegistry.createUser("example@gmail.com", "123456")

User Email: example@gmail.com - Password: b'$2b$12$/DCpnsvY1S30SHPVn6UXIec4B6MfjtYU2mveF4bVn/LK.IYs8/Jiu' has been saved on the database.


The system works. 

However, the encryption logic is inside the UserRegistry class, exceeding its funcionality: UserRegistry is a class which deals with registring new users, not encrypting their passwords. If you change or extend the encryption algorithm, you will have to modify this class and maybe other parts of your system that perform encryption.

- Satisfying the SRP:

A better solution is to have another class that deals with all the encryption logic of the system. 
In that way, we satisfy the SRP:

In [7]:

import bcrypt

# This class represents an user
class User:
    def __init__(self, email: str, password: str) -> None:
        self.email = email
        self.password = password
        # You can add here more info about the user...
        
    def __str__(self) -> str:
        return f"Email: {self.email} - Password: {self.password}"
        
# This class manages database operations
class DatabaseManager:
    
    # Here you will have tons of methods to manage database operations...
    
    @staticmethod
    def save_to_database(user: User) -> None:
        print(f"User {str(user)} has been saved on the database.")

# We create here a new class to manage all the encryption logic
class Encrypter:
    
    @staticmethod
    def encrypt(password: str) -> bytes:
        return bcrypt.hashpw(password.encode(), bcrypt.gensalt())

# And finally here is a class that registers a user in the database
class UserRegistry:
    
    @staticmethod
    def createUser(email: str, password: str) -> None:
        """Method for creating a new user."""
        
        # First, we have to encrypt the user's password
        encrypted_password = Encrypter.encrypt(password)
        
        # Then, we create the User object
        user = User(email, encrypted_password)
        
        # And finally we send it to the database
        DatabaseManager.save_to_database(user)
        
if  __name__ == "__main__":
    
    # Let's create and register a new user
    UserRegistry.createUser("example@gmail.com", "123456")

User Email: example@gmail.com - Password: b'$2b$12$la7tELE74uWmY95FdS4ZY.i/MmoLA8YVlPgTn0cvhZbav5dTPghqy' has been saved on the database.


It works too, but in this case if we need to change something related to the encryption algorithm, we will have to change the Encrypter class while the others will remain unmodified. That makes much more sense.

To sum up: **One class -> One purpose**

## 2. Open Closed Principle (OCP)

The Open Closed Principle stands that a software entity should be open to its extension but closed to its modification.

What this means is that if we want to add a new functionality, then we should write new code instead of modifying new one.
Preventing bugs is much more easy in this way.

### Example: Computing the area of some polygons.

- Without satisfying the OCP:

In [9]:
from typing import List

# Let's define a Rectangle class
class Rectangle:
    
    def __init__(self, width: float, height: float) -> None:
        self.width = width
        self.height = height
        

# Now let's create another class to compute areas
class AreaCalculator:
    
    @staticmethod
    def compute_area(shapes: List[Rectangle]) -> float:
        """Computes the area of a list of shapes."""
        
        area = 0
        for shape in shapes:
            area = area + shape.width * shape.height
        return area
    
if __name__ == "__main__":
    
    list_of_rectangles = [Rectangle(5, 6), Rectangle(4, 5)]
    total_area = AreaCalculator.compute_area(list_of_rectangles)
    print(f"Total area: {total_area}")

Total area: 50


Now let's say we want compute the area of triangles as well, that is, we want to add a new functionality to our software.
Following the previous code, the way to go would be:

In [20]:
from typing import List

# Let's define a Rectangle class
class Rectangle:
    
    def __init__(self, width: float, height: float) -> None:
        self.width = width
        self.height = height
        
class Triangle:
    def __init__(self, width: float, height: float) -> None:
        self.width = width
        self.height = height

# Now let's create another class to compute areas
class AreaCalculator:
    
    @staticmethod
    def compute_area(shapes: list) -> float:
        """Computes the area of a list of shapes."""
        
        area = 0
        for shape in shapes:
            # Here we are modifying the code!!
            if isinstance(shape, Rectangle):
                area = area + shape.width * shape.height
            elif isinstance(shape, Triangle):
                area = area + (shape.width * shape.height / 2)
        return area
    
if __name__ == "__main__":
    
    list_of_rectangles = [Rectangle(5, 6), Rectangle(4, 5), Triangle(4, 5)]
    total_area = AreaCalculator.compute_area(list_of_rectangles)
    print(f"Total area: {total_area}")

Total area: 60.0


To add the new functionality, we had to modify our compute_area() method, making much easier to add bugs.

- Satisfying the OCP:

To satisfy the OCP, we can use inheritance and polymorphism. Let's see how:

In [21]:

from abc import ABC, abstractmethod

# First, we define a interface as an abstract class that works as the
# base class for every single shape we want to create, like rectangles,
# triangles and so on.
class IShape(ABC):
    
    # We define an abstract method as well, the one that deals with the
    # calculation of the area for each shape.
    @abstractmethod
    def compute_area(self) -> float:
        """Computes the shape's area."""
        
# Now, we create our first shape: a rectangle
class Rectangle(IShape):  # we inherit the rectangle from the IShape class
    
    def __init__(self, width: int, height: int) -> None:
        self.width = width
        self.height = height
        
    # Now, we override the abstract method from the parent class and 
    # implement the calculation of the area for a triangle
    def compute_area(self) -> float:
        return self.width * self.height 
    
# Let's do the same with the triangle class
class Triangle(IShape):  # again we inherit from IShape
    
    def __init__(self, width: int, height: int) -> None:
        self.width = width
        self.height = height
        
    # Now, we override the abstract method from the parent class and 
    # implement the calculation of the area for a triangle
    def compute_area(self) -> float:
        return (self.width * self.height / 2)

# Let's define again our area claculator class
class AreaCalculator:
    
    # Now this method remains the same whatever shape we inject to it
    @staticmethod
    def compute_area(shapes: List[IShape]) -> float:
        """Computes the area of a list of shapes."""
        
        area = 0
        for shape in shapes:
            area = area + shape.compute_area()
        return area
    
if __name__ == "__main__":
    
    list_of_rectangles = [Rectangle(5, 6), Rectangle(4, 5), Triangle(4, 5)]
    total_area = AreaCalculator.compute_area(list_of_rectangles)
    print(f"Total area: {total_area}")

Total area: 60.0


That's the way to go. We could continue defining new shapes and the core code would not change.

To sum up: **New functionality -> New code**

## 3. Liskov Substitution Principle (LSP)

This principle stands that every child class can be used in the same way as its parent class. 

That means that we can change both classes and our system will continue working properly.

### Example: 

- Without satisfying the LSP:

In [25]:

# Let's define a class that represents a real Duck.
class Duck:
    
    # The duck can do some actions such as swim, fly and cuack.
    def swim(self) -> None:
        print("The duck is swimming!")
        
    def fly(self) -> None:
        print("The duck is flying!")
        
    def cuack(self) -> None:
        print("Duck is saying: Cuack!")
        
# Now we inherit from Duck a RubberDuck
class RubberDuck(Duck):
    
    # The rubber duck can swim and cuack but it cannot fly, so we raise 
    # and exception if trying to.
    def swim(self) -> None:
        print("The rubber duck is swimming!")
        
    def fly(self) -> None:
        raise Exception("Rubber duck cannot fly!")
    
    def cuack(self) -> None:
        print("Rubber duck is saying: Cuack!")
    
class DuckProcesser:
    
    @staticmethod
    def make_ducks_fly(ducks: List[Duck]) -> None:
        for duck in ducks:
            try:
                duck.fly()
            except Exception as e:
                raise e
            
if __name__ == "__main__":
    
    DuckProcesser.make_ducks_fly([Duck(), RubberDuck()])

The duck is flying!


Exception: Rubber duck cannot fly!

- Satisfying the LSP:

The solution would be to re-think the design: we are going to define individual components or interfaces and make both Duck and RubberDuck implement some of them
instead of using inheritance. 

In [27]:

# Let's define the interfaces for swim, fly and cuack.
from abc import ABC, abstractmethod

class ISwim(ABC):
    
    @abstractmethod
    def swim():
        """Swims"""
        
class IFly(ABC):
    
    @abstractmethod
    def fly():
        """Flies"""
        
class ICuack(ABC):
    
    @abstractmethod
    def cuack():
        """Cuacks"""


# Again, let's define a class that represents a real Duck. A Duck implements
# the three interfaces: ISwim, IFly and ICuack.
class Duck:
    
    # The duck can do some actions such as swim, fly and cuack.
    def swim(self) -> None:
        print("The duck is swimming!")
        
    def fly(self) -> None:
        print("The duck is flying!")
        
    def cuack(self) -> None:
        print("Duck is saying: Cuack!")
        
# Now we create a RubberDuck class but instead of deriving the Duck class,
# we implement two interfaces: ISwim and ICuack.
class RubberDuck:
    def swim(self) -> None:
        print("The rubber duck is swimming!")
    
    def cuack(self) -> None:
        print("Rubber duck is saying: Cuack!")
    
class DuckProcesser:
    
    @staticmethod
    def make_ducks_fly(ducks: List[Duck]) -> None:
        # We cannot pass rubber ducks here!
        for duck in ducks:
            try:
                duck.fly()
            except Exception as e:
                raise e
            
if __name__ == "__main__":
    
    DuckProcesser.make_ducks_fly([Duck()])

The duck is flying!


PS. In Python, it's not neccessary to declare the interfaces ISwim, IFly and ICuack. You just can create both Duck and RubberDuck classes and avoid the inheritance to satisfy the LSP.

To sum up: **A child could replaced its parent in the system and viceversa.**

## 4. Interface Segregation Principle (ISP)

This principle states that a class or interface should know only those methods it really uses, and it should not depend on those it does not need to use or are used rarely in specific parts of the code. ISP splits interfaces that are very large into smaller and more specific ones so that they only have to know about the methods that are of interest to them.

In other words, this means that it's usually better to have multiple, smaller and specialized classes or interfaces with specific methods than having a big one with many methods that are rarely used. Otherwise, adding new functionalities would become more difficult each time. 

### Example: 

Let's imagine we have two types of workers in a company: Human workers and human robots. Of course, appart from working these two types can do different activities such as lunch in the case of humans or recharge in the case of robots.

- Without satisfying the ISP:

In [3]:
from abc import ABC, abstractmethod

class IWorker(ABC):
        
    @abstractmethod
    def work():
        """Works"""
        
    @abstractmethod
    def lunch():
        """Break for a lunch"""
        
    @abstractmethod
    def recharge():
        """Recharges batteries"""
        
# If we derive a class which represents a concrete role of a human worker  
# from IWorker, it will inherit the 'recharge()' method as well. This is 
# unnecessary and it does not satisfy the ISP.
class Manager(IWorker):
    
    def work(self):
        print("I'm working!")
        
    def lunch(self):
        print("I'm having lunch")
        
    def recharge(self):
        print("It makes no sense for a human to recharge!!")
    

# Similarly, if we derive a class to represent a concrete type of robot,
# it will inherit the 'lunch()' method which is unnecessary to be known
# by a robot.
class Machine(IWorker):
    
    def work(self):
        print("I'm working!")
        
    def lunch(self):
        print("It makes no sense for a robot to have lunch!!")
        
    def recharge(self):
        print("I'm recharging...")

- Satisfying the ISP: Instead of having one unique interface called IWorker, we are going to design our problem using 3 interfaces: one common interface IWorker, which contains all the methods used by both the human worker and the robot worker, a specific interface IHuman, which contains all the methods used only by the human workers, and similarly another one called IRobot with methods used by only the robots.

In [2]:

from abc import ABC, abstractmethod

class IWorker(ABC):
        
    @abstractmethod
    def work():
        """Works"""
        
class IHuman(ABC):
        
    @abstractmethod
    def lunch():
        """Break for a lunch"""

class IRobot(ABC):
    
    @abstractmethod
    def recharge():        
        """Recharges batteries"""

# Now, we inherint a human worker from both IWorker and IHuman
class Manager(IWorker, IHuman):
    
    def work(self):
        print("I'm working!")
        
    def lunch(self):
        print("I'm having lunch!")
        
# And we inherit a robot worker from both IWorker and IRobot
class Machine(IWorker, IRobot):
    def work(self):
        print("I'm working!")
        
    def recharge(self):
        print("I'm recharging...")

To sum up: **Interfaces and classes should be specilized and they should only know about methods of their interest.** 

## 5. Dependency Inversion Principle (DIP)

This principle states that high-level modules should not import anything from low-level modules; Both should depend on abstractions like interfaces. In that way, we deacouple the different software modules our system has.

For example, following the DIP we do not depend on the technology used by a database communication; We only depend on abstract layers like interfaces, that is, the components of our system communicate with each other by interfaces and they do not take into account if we are using, for example, PosgreSQL or MongoDB. At a time, we could change PosgreSQL to MongoDB or viceversa and the rest of the components would remain unmodified. 

### Example: 

- Satisfying DIP:

In [3]:
from typing import List

# We are going to import SpaCy to create a tokenizer. 
# Make sure to make 'pip install spacy' to install SpaCy and 'python -m 
# spacy download [model_name]' to download the appropiate model. 
# Visit https://spacy.io/usage for more info.
import spacy

class Tokenizer:
    
    def __init__(self, model: str = "en_core_web_sm") -> None:
        """Constructor. Loads the model. """
        self._model = spacy.load(model)
    
    def tokenize(self, sentence: str) -> List[str]:
        """Tokenizes a sentence and returns the tokens."""
        doc = self._model(sentence)
        return [token.text for token in doc]

if __name__ == '__main__':
    
    # We create the Tokenizer instance, an example of sentence and do
    # tokenization
    tokenizer = Tokenizer()
    example = "This is an example of sentence"
    tokens = tokenizer.tokenize(example)
    print(f"Sentence '{example}' has been tokenized to '{tokens}'.")
        

Sentence 'This is an example of sentence' has been tokenized to '['This', 'is', 'an', 'example', 'of', 'sentence']'.


As you can see, the previous example works fine. 

Now imagine that for some reason we decide to use 'nltk' instead of SpaCy. 
Here's the new Tokenizer class:

In [7]:
from typing import List

# We are going to import nltk now. Make sure to do 'pip install nltk'.
# More info: https://www.nltk.org/install.html
import nltk


class Tokenizer:

    # Here, we are changing the constructor method to use nltk
    def __init__(self, model: str = "punkt") -> None:
        """Constructor. Loads the model."""
        nltk.download(model)

    # And we are changing the tokenize() method as well.
    def tokenize(self, sentence: str) -> List[str]:
        """Tokenizes a sentence and returns the tokens."""
        return nltk.word_tokenize(sentence)


if __name__ == "__main__":

    # However, this remains the same. We instanciate a Tokenizer, create
    # a sentence and do tokenization. This happens because we are relying
    # on interface, a constructor method and a method called tokenize()
    # from the Tokenizer class. We do not care about how the Tokenizer is
    # built not about how the tokenization is done.

    # Apart from main(), this section of code could be placed on another
    # script or on another class... This is where DIP makes even more
    # sense
    tokenizer = Tokenizer()
    example = "This is an example of sentence"
    tokens = tokenizer.tokenize(example)
    print(f"Sentence '{example}' has been tokenized to '{tokens}'.")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ana\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


Sentence 'This is an example of sentence' has been tokenized to '['This', 'is', 'an', 'example', 'of', 'sentence']'.


*Source: BettaTech - https://www.youtube.com/watch?v=2X50sKeBAcQ*